to do

1. change 'scheduler' => 'schedule'
2. install git in the mothership directory
3. push to github
4. get docker working
5. install on pi

docker build -t pyrambium .

docker run -it --rm -p 8000:8000 -v $PYRAMBIUM_SAVED_DIR:$PYRAMBIUM_SAVED_DIR -v $PYRAMBIUM_OUTPUT_DIR:$PYRAMBIUM_OUTPUT_DIR --name pyrambium_running pyrambium

In [ ]:
# docker run -it --rm -p 8000:8000 -v $PYRAMBIUM_SAVED_DIR:$PYRAMBIUM_SAVED_DIR -v $PYRAMBIUM_OUTPUT_DIR:$PYRAMBIUM_OUTPUT_DIR --name pyrambium_running pyrambium

python3 -m compileall -f base

to run at the command line in the python/...

uvicorn main:app --workers 1

uvicorn main:app --reload

import uvicorn
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

pip3 install pylint

pylint *.py

When you run a notebook cell that imports from the Pyrambium packages, you need to set the NOTEBOOK FILE ROOT to \$\{workspaceFolder\} so  that imports from top-level packages work. For example, with the property's default value of \$\{fileDirName\}, the imports from mothership.action will fail since the notebook location is one directory down from top-level.





In [1]:
from datetime import time
from pydantic import BaseModel
import requests
from whendo.core.scheduler import TimelyScheduler, RandomlyScheduler
import whendo.core.actions.file_action as file_x
import whendo.core.actions.gpio_action as gpio_x
from whendo.core.util import PP, TimeUnit, Dirs

prefix = "http://127.0.0.1:5000"
prefix = "http://192.168.0.45:8000"
prefix = "http://locahost:8000"
prefix = "http://localhost:8000"
def get(path:str):
    cmd = f"{prefix}{path}"
    response = requests.get(cmd)
    return response.json()
def put(path:str, data:BaseModel):
    cmd = f"{prefix}{path}"
    response = requests.put(cmd, data.json())
    PP.pprint(response.json())
def put_as_is(path:str, data:None):
    cmd = f"{prefix}{path}"
    response = requests.put(cmd, data)
    PP.pprint(response.json())
def delete(path:str):
    cmd = f"{prefix}{path}"
    response = requests.delete(cmd)
    return response.json()
output_dir = Dirs.output_dir()

In [2]:
Dirs.output_dir()

'/Users/electronhead/whendo/output/'

In [12]:
get('/mothership/load')

{'actions': {'heartbeat1': {'file': '/Users/electronhead/whendo/output/heartbeat1.txt',
   'xtra': {'words': 'heartbreak hotel'}},
  'heartbeat2': {'file': '/Users/electronhead/whendo/output/heartbeat2.txt',
   'xtra': {'words': 'nothing but heartaches'}},
  'heartbeat3': {'file': '/Users/electronhead/whendo/output/heartbeat3.txt',
   'xtra': {'words': 'heart of glass'}},
  'green_on': {'pin': 27, 'on': True},
  'green_off': {'pin': 27, 'on': False},
  'green_toggle': {'pin': 27},
  'gpio_clear': {}},
 'schedulers': {'randomly_soon': {'start': None,
   'stop': None,
   'time_unit': 'second',
   'low': 10,
   'high': 20},
  'timely_day': {'start': '06:00:00',
   'stop': '18:00:00',
   'interval': 1,
   'hour': None,
   'minute': None,
   'second': None},
  'timely_night': {'start': '18:00:00',
   'stop': '06:00:00',
   'interval': 1,
   'hour': None,
   'minute': None,
   'second': None}},
 'schedulers_actions': {'randomly_soon': ['heartbeat3'],
  'timely_day': ['heartbeat1'],
  'timely

In [13]:
get('/mothership/clear')

'mothership cleared'

In [10]:
Dirs.output_dir()

'/Users/electronhead/pyrambium/output/'

In [14]:
action = file_x.FileHeartbeat(file=f"{output_dir}heartbeat1.txt", xtra={'words':'heartbreak hotel'})
put('/actions/heartbeat1', action)
action = file_x.FileHeartbeat(file=f"{output_dir}heartbeat2.txt", xtra={'words':'nothing but heartaches'})
put('/actions/heartbeat2', action)
action = file_x.FileHeartbeat(file=f"{output_dir}heartbeat3.txt", xtra={'words':'heart of glass'})
put('/actions/heartbeat3', action)
put('/actions/green_on', gpio_x.SetPin(pin=27, on=True))
put('/actions/green_off', gpio_x.SetPin(pin=27, on=False))
put('/actions/green_toggle', gpio_x.TogglePin(pin=27))
put('/actions/gpio_clear', gpio_x.Cleanup())

scheduler = RandomlyScheduler(time_unit=TimeUnit.second, low=10, high=20)
put('/schedulers/randomly_soon', scheduler)

morning, evening = time(6,0,0), time(18,0,0)
scheduler = TimelyScheduler(interval=1, start=morning, stop=evening)
put('/schedulers/timely_day', scheduler)
scheduler = TimelyScheduler(interval=1, start=evening, stop=morning)
put('/schedulers/timely_night', scheduler)

get('/schedulers/timely_day/actions/heartbeat1')
get('/schedulers/timely_night/actions/heartbeat2')
get('/schedulers/randomly_soon/actions/heartbeat3')

'action (heartbeat1) was successfully added'
'action (heartbeat2) was successfully added'
'action (heartbeat3) was successfully added'
'action (green_on) was successfully added'
'action (green_off) was successfully added'
'action (green_toggle) was successfully added'
'action (gpio_clear) was successfully added'
'scheduler (randomly_soon) was successfully added'
'scheduler (timely_day) was successfully added'
'scheduler (timely_night) was successfully added'


'action (heartbeat3) was successfully scheduled (randomly_soon)'

In [5]:
get('/mothership/load')

{'actions': {'heartbeat1': {'file': '/Users/electronhead/whendo/output/heartbeat1.txt',
   'xtra': {'words': 'heartbreak hotel'}},
  'heartbeat2': {'file': '/Users/electronhead/whendo/output/heartbeat2.txt',
   'xtra': {'words': 'nothing but heartaches'}},
  'heartbeat3': {'file': '/Users/electronhead/whendo/output/heartbeat3.txt',
   'xtra': {'words': 'heart of glass'}},
  'green_on': {'pin': 27, 'on': True},
  'green_off': {'pin': 27, 'on': False},
  'green_toggle': {'pin': 27},
  'gpio_clear': {}},
 'schedulers': {'randomly_soon': {'start': None,
   'stop': None,
   'time_unit': 'second',
   'low': 10,
   'high': 20},
  'timely_day': {'start': '06:00:00',
   'stop': '18:00:00',
   'interval': 1,
   'hour': None,
   'minute': None,
   'second': None},
  'timely_night': {'start': '18:00:00',
   'stop': '06:00:00',
   'interval': 1,
   'hour': None,
   'minute': None,
   'second': None}},
 'schedulers_actions': {'randomly_soon': ['heartbeat3'],
  'timely_day': ['heartbeat1'],
  'timely

In [ ]:
action = file_x.FileHeartbeat(file=f"{output_dir}heartbeat3.txt")
put('/actions/heartbeat3', action)
get('/schedulers/timely_day/actions/heartbeat3')

In [ ]:
action = file_x.FileHeartbeat(file=f"{output_dir}heartbeat4.txt", xtra={'words':"your cheatin' heart"})
put('/actions/heartbeat4', action)
get('/schedulers/timely_day/actions/heartbeat4')

In [15]:
get('/jobs/start')

'started running'

In [16]:
get('/jobs/stop')

'stopped running'

In [10]:
get('/jobs/count')

{'job_count': 3}

In [ ]:
get('/actions/heartbeat1/execute')

In [ ]:
get('/actions/heartbeat2/execute')

In [ ]:
get('/actions/heartbeat3/execute')

In [ ]:
get('/actions/heartbeat4/execute')

In [ ]:
get('/actions/heartbeat1')

In [ ]:
get('/actions/heartbeat2')

In [ ]:
get('/actions/heartbeat1')

In [ ]:
get('/schedulers/timely_day')

In [ ]:
get('/mothership/load')['schedulers_actions']

In [ ]:
get('/mothership/load')['schedulers']

In [ ]:
get('/mothership/load')

In [ ]:
delete('/actions/green_toggle')

In [ ]:
put('/actions/green_toggle', TogglePin(pin=27))

In [ ]:
get('/actions/green_on/execute')

In [ ]:
get('/actions/green_off/execute')

In [ ]:
get('/actions/green_toggle/execute')

In [ ]:
get('/actions/gpio_clear')